### Managing Nulls 

In [55]:
import pandas as pd
from numpy import random

In [56]:
df = pd.read_csv(r'C:\Users\risha\Documents\KRMU\AIML_assigment\datasets\iot_example_with_nulls.csv')

### Data Quality Check

In [57]:
df.head()

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,NaN,NaN


In [58]:
df.dtypes

timestamp       object
username        object
temperature    float64
heartrate        int64
build           object
latest         float64
note            object
dtype: object

In [59]:
df.note.value_counts()

note
wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: count, dtype: int64

### Let's remove all null values (including the note: n/a)

In [60]:
df = pd.read_csv(r'C:\Users\risha\Documents\KRMU\AIML_assigment\datasets\iot_example_with_nulls.csv', na_values=['n/a'])

### Test to see if we can use dropna

In [61]:
df.shape

(146397, 7)

In [62]:
df.dropna().shape

(46116, 7)

In [63]:
df.dropna(how='all', axis=1).shape

(146397, 7)

### Test to see if we can drop columns

In [64]:
my_columns = list(df.columns)

In [65]:
my_columns

['timestamp',
 'username',
 'temperature',
 'heartrate',
 'build',
 'latest',
 'note']

In [66]:
list(df.dropna(thresh=int(df.shape[0] * .9), axis=1).columns)

['timestamp', 'username', 'heartrate']

### I want to find all columns that have missing data

In [67]:
missing_info = list(df.columns[df.isnull().any()])

In [68]:
missing_info

['temperature', 'build', 'latest', 'note']

In [69]:
for col in missing_info:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))

number missing for column temperature: 32357
number missing for column build: 32350
number missing for column latest: 32298
number missing for column note: 48704


In [70]:
for col in missing_info:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

percent missing for column temperature: 0.22102228870810195
percent missing for column build: 0.22097447352063226
percent missing for column latest: 0.22061927498514314
percent missing for column note: 0.332684412931959


### Can I easily substitute majority values in for missing data?

In [71]:
df.note.value_counts()

note
wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: count, dtype: int64

In [72]:
df.build.value_counts().head()

build
4e6a7805-8faa-2768-6ef6-eb3198b483ac    1
12aefc6b-272c-751e-6117-134ee73e2649    1
fd4049c3-2297-14ac-a27e-6da57129dd10    1
0bcfab8f-bc25-3f8f-8585-0614e1555fd1    1
b0de05dd-2860-abbb-8be6-f5c0e30ca063    1
Name: count, dtype: int64

In [73]:
df.latest.value_counts()

latest
0.0    75735
1.0    38364
Name: count, dtype: int64

In [74]:
df.latest = df.latest.fillna(0)

### Have not yet addressed temperature missing values... Let's find a way to fill

In [75]:
df.username.value_counts().head()

username
esmith    45
zsmith    43
vsmith    41
ysmith    40
jsmith    37
Name: count, dtype: int64

In [76]:
df = df.set_index('timestamp')

In [77]:
df.head()

,username,temperature,heartrate,build,latest,note
timestamp,,,,,,
2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN


In [78]:
df.temperature = df.groupby(df['username']).temperature.fillna(df.temperature.mean())

### Exercise: How many temperature values did I fill? What percentage of values are still missing (for temperature)?

In [79]:
rows_filled = 32357 - df[df.temperature.isnull() == True].shape[0] 
still_missing = df[df.temperature.isnull() == True].shape[0] / df.shape[0]


In [80]:
rows_filled

32357

In [81]:
still_missing

0.0